In [13]:
import pandas as pd
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
import numpy as np
from sklearn.ensemble import IsolationForest
import ppscore as pps
df = pd.read_csv('/content/adult_with_headers.csv')
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32561 entries, 0 to 32560
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   age             32561 non-null  int64 
 1   workclass       32561 non-null  object
 2   fnlwgt          32561 non-null  int64 
 3   education       32561 non-null  object
 4   education_num   32561 non-null  int64 
 5   marital_status  32561 non-null  object
 6   occupation      32561 non-null  object
 7   relationship    32561 non-null  object
 8   race            32561 non-null  object
 9   sex             32561 non-null  object
 10  capital_gain    32561 non-null  int64 
 11  capital_loss    32561 non-null  int64 
 12  hours_per_week  32561 non-null  int64 
 13  native_country  32561 non-null  object
 14  income          32561 non-null  object
dtypes: int64(6), object(9)
memory usage: 3.7+ MB
None


In [14]:
print(df.head())

   age          workclass  fnlwgt   education  education_num  \
0   39          State-gov   77516   Bachelors             13   
1   50   Self-emp-not-inc   83311   Bachelors             13   
2   38            Private  215646     HS-grad              9   
3   53            Private  234721        11th              7   
4   28            Private  338409   Bachelors             13   

        marital_status          occupation    relationship    race      sex  \
0        Never-married        Adm-clerical   Not-in-family   White     Male   
1   Married-civ-spouse     Exec-managerial         Husband   White     Male   
2             Divorced   Handlers-cleaners   Not-in-family   White     Male   
3   Married-civ-spouse   Handlers-cleaners         Husband   Black     Male   
4   Married-civ-spouse      Prof-specialty            Wife   Black   Female   

   capital_gain  capital_loss  hours_per_week  native_country  income  
0          2174             0              40   United-States   <=50

In [15]:
print(df.describe())


                age        fnlwgt  education_num  capital_gain  capital_loss  \
count  32561.000000  3.256100e+04   32561.000000  32561.000000  32561.000000   
mean      38.581647  1.897784e+05      10.080679   1077.648844     87.303830   
std       13.640433  1.055500e+05       2.572720   7385.292085    402.960219   
min       17.000000  1.228500e+04       1.000000      0.000000      0.000000   
25%       28.000000  1.178270e+05       9.000000      0.000000      0.000000   
50%       37.000000  1.783560e+05      10.000000      0.000000      0.000000   
75%       48.000000  2.370510e+05      12.000000      0.000000      0.000000   
max       90.000000  1.484705e+06      16.000000  99999.000000   4356.000000   

       hours_per_week  
count    32561.000000  
mean        40.437456  
std         12.347429  
min          1.000000  
25%         40.000000  
50%         40.000000  
75%         45.000000  
max         99.000000  


In [16]:
print(df.isnull().sum())


age               0
workclass         0
fnlwgt            0
education         0
education_num     0
marital_status    0
occupation        0
relationship      0
race              0
sex               0
capital_gain      0
capital_loss      0
hours_per_week    0
native_country    0
income            0
dtype: int64


In [17]:
num_features = df.select_dtypes(include=['int64', 'float64']).columns
scaler = StandardScaler()
df[num_features] = scaler.fit_transform(df[num_features])
min_max_scaler = MinMaxScaler()
df[num_features] = min_max_scaler.fit_transform(df[num_features])


In [18]:
cat_features = df.select_dtypes(include=['object']).columns
for feature in cat_features:
    if df[feature].nunique() < 5:
        df = pd.get_dummies(df, columns=[feature], drop_first=True)


In [19]:
le = LabelEncoder()
cat_features = df.select_dtypes(include=['object']).columns
for feature in cat_features:
    if df[feature].nunique() >= 5:
        df[feature] = le.fit_transform(df[feature])

In [20]:
df['age_income_ratio'] = df['age'] / df['hours_per_week']


In [21]:
import numpy as np

df['education'] = df['education'].astype(str)

df['education_level'] = df['education'].apply(lambda x: len(x.split()))

df['log_hours_per_week'] = np.log1p(df['hours_per_week'])

print(df.head())

        age  workclass    fnlwgt education  education_num  marital_status  \
0  0.301370          7  0.044302         9       0.800000               4   
1  0.452055          6  0.048238         9       0.800000               2   
2  0.287671          4  0.138113        11       0.533333               0   
3  0.493151          4  0.151068         1       0.400000               2   
4  0.150685          4  0.221488         9       0.800000               2   

   occupation  relationship  race  capital_gain  capital_loss  hours_per_week  \
0           1             1     4       0.02174           0.0        0.397959   
1           4             0     4       0.00000           0.0        0.122449   
2           6             1     4       0.00000           0.0        0.397959   
3           6             0     2       0.00000           0.0        0.397959   
4          10             5     2       0.00000           0.0        0.397959   

   native_country  sex_ Male  income_ >50K  age_in

In [22]:
import numpy as np
df['log_hours_per_week'] = np.log1p(df['hours_per_week'])


In [23]:
import numpy as np
from sklearn.ensemble import IsolationForest

df['age_income_ratio'] = df.apply(lambda row: row['age'] / row['hours_per_week'] if row['hours_per_week'] != 0 else 0, axis=1)

df['log_hours_per_week'] = np.log1p(df['hours_per_week'].apply(lambda x: max(x, 1)))
iso_forest = IsolationForest(contamination=0.01)
outliers = iso_forest.fit_predict(df.select_dtypes(include=['int64', 'float64']))
df = df[outliers == 1]

In [24]:
pps_matrix = pps.matrix(df)
print(pps_matrix)


                      x                   y   ppscore                case  \
0                   age                 age  1.000000      predict_itself   
1                   age           workclass  0.000000          regression   
2                   age              fnlwgt  0.000000          regression   
3                   age           education  0.068677      classification   
4                   age       education_num  0.000000          regression   
..                  ...                 ...       ...                 ...   
319  log_hours_per_week           sex_ Male  0.000000          regression   
320  log_hours_per_week        income_ >50K  0.000000          regression   
321  log_hours_per_week    age_income_ratio  0.000000          regression   
322  log_hours_per_week     education_level  0.000000  target_is_constant   
323  log_hours_per_week  log_hours_per_week  1.000000      predict_itself   

     is_valid_score               metric  baseline_score  model_score  \
0 